In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()   

[ Using Seed :  42  ]


In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils

### Handle Function

In [4]:
#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
def create_dataloaders(CONFIG, X, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

### Create models

**Conv + MLP**

In [5]:
class Block(nn.Module):
    def __init__(self, inp, out, ks):
        super().__init__()
        self.activ = F.gelu
        self.conv1 = nn.Conv1d(inp, out, ks, padding=1)
        self.bn1 = nn.BatchNorm1d(out)

    def forward(self, x):
        x = self.activ(self.bn1(self.conv1(x)))
        return x

class ConvMlpNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.h_dims = [1, 32, 64, self.h_dim]
        self.backbone = nn.Sequential(*[Block(self.h_dims[i], self.h_dims[i + 1], 3) for i in range(len(self.h_dims) - 1)])
        self.flat = nn.Flatten()
        self.drop = nn.Dropout(p=self.drop_p)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.lin = nn.Sequential(nn.Linear(self.h_dim, self.h_dim * 2),
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4))

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flat(x)
        x = self.drop(x)
        x = self.lin(x)
        return x

**Only MLP**

In [6]:
class MlpNet(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 13),
                                 nn.BatchNorm1d(in_features * 13),
                                 nn.GELU(),
                                 nn.Linear(in_features * 13, self.h_dim),
                                 nn.BatchNorm1d(self.h_dim),
                                 nn.GELU(),
                                 nn.Linear(self.h_dim, self.h_dim * 2), 
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x

### Trainable params comparing

In [7]:
cnn_mlp = ConvMlpNet()
mlp = MlpNet(16)

utils.count_params(cnn_mlp), utils.count_params(mlp) 

[INFO]: Model "ConvMlpNet" has 228412 trainable parameters
[INFO]: Model "MlpNet" has 229676 trainable parameters


(None, None)

### Time comparation

In [8]:
from time import time

data_path = p_join('..', '..', 'data_2')
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=10.6' in item]


CONFIG = {'train_batch_size': 128,
          'test_batch_size': 128,
          'cpu_workers': 2}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
X = np.expand_dims(X, axis=1)
train_dataloader_cnn, _ = create_dataloaders(CONFIG, X, Y)

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
train_dataloader_mlp, _ = create_dataloaders(CONFIG, X, Y)


#########################
## SWITCH TO EVAL MODE ##
#########################
cnn_mlp = cnn_mlp.eval()
mlp = mlp.eval()

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 22.11it/s]


In [9]:
x, y = next(iter(train_dataloader_cnn))

In [10]:
%%timeit
with torch.no_grad():
    out = cnn_mlp(x)

5.32 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
x, y = next(iter(train_dataloader_mlp))

In [12]:
%%timeit
with torch.no_grad():
    out = mlp(x)

1.93 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
%%timeit
for _ in range(2):
    for x, y in train_dataloader_cnn:
        with torch.no_grad():
            out = cnn_mlp(x)

6.6 s ± 165 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
for _ in range(2):
    for x, y in train_dataloader_mlp:
        with torch.no_grad():
            out = mlp(x)

3.32 s ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
from sklearn.model_selection import train_test_split
from sklearn import svm


model = svm.SVC(C=1000)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model.fit(X_train[::30], Y_train[::30])

SVC(C=1000)

In [22]:
%%timeit
for _ in range(2):
    for i in range(399):
        model.predict(X_test[33: 33 + CONFIG['test_batch_size']])

7.19 s ± 572 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
CONFIG = {'ephs': 35,
          'train_batch_size': 64,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNet(16)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|█████████████████████████████████████████| 799/799 [01:35<00:00,  8.32it/s]


Epoch: 1/35, 	 total train loss: 1.013586215217958


100%|█████████████████████████████████████████| 800/800 [00:14<00:00, 54.43it/s]


Epoch: 1/35, 	 total score test: 56.825818551222945, [best score: 56.825818551222945]



100%|█████████████████████████████████████████| 799/799 [00:39<00:00, 20.04it/s]


...

Epoch: 34/35, 	 total score test: 79.6124091583965, [best score: 79.6124091583965]



100%|█████████████████████████████████████████| 799/799 [00:09<00:00, 88.77it/s]


Epoch: 35/35, 	 total train loss: 0.43275033965128684


100%|████████████████████████████████████████| 800/800 [00:02<00:00, 366.44it/s]

Epoch: 35/35, 	 total score test: 79.12010627490818, [best score: 79.6124091583965]



In [60]:
utils.count_params(net)
pass

[INFO]: Model "MlpNet_s" has 20068 trainable parameters


In [85]:
class MlpNeLight(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.drop_p = 0.1
        self.drop = nn.Dropout(p=self.drop_p)
        self.lin = nn.Sequential(
                                 nn.Linear(in_features, in_features * 8),
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Linear(in_features * 8, in_features * 32),
                                 nn.BatchNorm1d(in_features * 32),
                                 nn.GELU(),
                                 nn.Linear(in_features * 32, in_features * 8), 
                                 nn.BatchNorm1d(in_features * 8),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 8, in_features * 4),
                                 nn.BatchNorm1d(in_features * 4),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(in_features * 4, 4)
        )

    def forward(self, x):
        x = self.lin(x)
        return x


CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False}

X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)

net = MlpNeLight(16)
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 21.65it/s]


[INFO]: Model "MlpNet_s" has 144068 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:01<00:00, 25.26it/s]


Epoch: 1/100, 	 total train loss: 1.0941536365723123


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 303.37it/s]


...

Epoch: 99/100, 	 total score test: 81.50347737751035, [best score: 81.99578026099867]



100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 18.10it/s]


Epoch: 100/100, 	 total train loss: 0.23174853349218563


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 259.96it/s]

Epoch: 100/100, 	 total score test: 81.26123310150817, [best score: 81.99578026099867]



### Try extra featrues

In [120]:
from utils import dl_models

def create_dataset(folders, verbose=False, feature_nums=16):
    X_merged, Y_merged = [], []
    for folder_path in tqdm(folders):
        X_path = p_join(folder_path, '2nd_exp_Input.txt')
        Y_path_down = p_join(folder_path, '2nd_exp_Topology_down.txt')
        Y_path_up = p_join(folder_path, '2nd_exp_Topology_up.txt')
        params_path = p_join(folder_path, '2nd_exp_Parameters.txt')

        X = utils.select_n_center_features(pd.read_csv(X_path).values, feature_nums)
        X_params = pd.read_csv(params_path).values[:,:-5] # remove last values
        Y_down = pd.read_csv(Y_path_down).values
        Y_up = pd.read_csv(Y_path_up).values
        Y = np.array([utils.map_classes(Y_up[i], Y_down[i]) for i in range(Y_up.shape[0])])
        X = np.concatenate([X, X_params], axis=1)

        X_merged.append(X)
        Y_merged.append(Y)

    # Merge and shuffle it!
    X, Y = np.concatenate(X_merged, axis=0), np.concatenate(Y_merged, axis=0)
    indexes_for_shuffle = np.random.permutation(np.arange(X.shape[0]))
    X = X[indexes_for_shuffle]
    Y = Y[indexes_for_shuffle]
    if verbose:
        print('Dataset cteated!')
    return X, Y

X, Y = create_dataset(FOLDERS)
X = utils.normalize_data(X)

100%|███████████████████████████████████████████| 16/16 [00:02<00:00,  7.08it/s]


In [121]:
X.shape

(63984, 17)

In [122]:
CONFIG['ephs'] = 10
net = dl_models.MlpNeLight(X.shape[1])
utils.count_params(net)
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

[INFO]: Model "MlpNeLight" has 162456 trainable parameters


100%|███████████████████████████████████████████| 49/49 [00:10<00:00,  4.88it/s]


Epoch: 1/10, 	 total train loss: 0.5395250259613504


100%|█████████████████████████████████████████| 200/200 [00:03<00:00, 55.95it/s]


Epoch: 1/10, 	 total score test: 77.62756896147535, [best score: 77.62756896147535]



...

Epoch: 9/10, 	 total score test: 96.73360943971244, [best score: 96.73360943971244]



100%|███████████████████████████████████████████| 49/49 [00:13<00:00,  3.72it/s]


Epoch: 10/10, 	 total train loss: 0.07644218464895171


100%|█████████████████████████████████████████| 200/200 [00:04<00:00, 41.94it/s]

Epoch: 10/10, 	 total score test: 96.6163944674533, [best score: 96.73360943971244]

